In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

pd.set_option('display.max_columns', None)

output = join("output","correcoes")

timer = Timer()

In [2]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

68.45608854293823

In [ ]:
casos = tb.get()
len(casos)

In [ ]:
duplicados = casos.loc[((casos.duplicated('hash')) & (casos['Obito'] == 'NAO'))].index.tolist()
duplicados_less = casos.loc[(casos['hash'].isin(casos['hash_less'])) & (casos['Obito'] == 'NAO')].index.tolist()
duplicados_more = casos.loc[(casos['hash'].isin(casos['hash_more'])) & (casos['Obito'] == 'NAO')].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('casos_duplicados.xlsx')
print(len(dropar))
casos = casos.drop(index=dropar)
len(casos)

In [ ]:
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('obitos_duplicados.xlsx')
len(dropar)

In [ ]:
writer = pd.ExcelWriter("novos_tb_pacientes.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.sort_values('Nome').to_excel(writer,index=None)

writer.save()
writer.close()